In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from procesadoDatosDemandaString import procesar_datos_string as procesar_datos_string_demanda 
from procesadoDatosPrecioString import procesar_datos_string as procesar_datos_string_precio
import matplotlib.pyplot as plt

In [2]:
precio = procesar_datos_string_precio()
demanda = procesar_datos_string_demanda()

Archivo donde están los datos del precio (sin extensión): precio
Archivo donde están los datos de la demanda (sin extensión): demanda


In [5]:
# Targets (precios sin sin septiembre de 2019)
y_train = precio.iloc[:len(precio)-122]
y_train = [x[i] for x in y_train.values.tolist() for i in range(len(x))]

# Datos de entrenamiento (demandas sin septiembre de 2019)
X_train = demanda.iloc[:len(precio)-122]
X_train = [x[i] for x in X_train.values.tolist() for i in range(len(x))]

# Normalizamos
media = np.array(X_train).mean(axis = 0)
desviacion = np.array(X_train).std(axis = 0)

X_train -= media
X_train/=desviacion

# Target_test (precios para septiembre de 2019)
y_test = precio[len(precio)-122:len(precio)-122 + 30]

In [6]:
# Datos de entrenamiento para pasar a la red neuronal

# Cada dato del conjunto tiene la demanda una hora y 24 horas antes
# para su precio correspondiente
X_train_2 = []
for i in range(24, len(y_train)):
    list_24_1_before = [X_train[i-24], X_train[i-1]]
    X_train_2.append(list_24_1_before)

# a los targets de entrenamiento le quitamos las primeras 24 horas    
y_train_2 = y_train[24:]

# Preparamos los datos para el test (estos son los datos que queremos predecir)

# Demanda de agosto
lista_agosto = ["201908" + str(i).zfill(2) for i in range(1, 31)]
demanda_agosto_2019 = demanda.loc[lista_agosto].values
demanda_agosto_2019 = [x[i] for x in demanda_agosto_2019 for i in range(len(x))]

# Demanda de septiembre
lista_septiembre = ["201909" + str(i).zfill(2) for i in range(1, 31)]
demanda_septiembre_2019 = demanda.loc[lista_septiembre].values
demanda_septiembre_2019 = [x[i] for x in demanda_septiembre_2019 for i in range(len(x))]

# Demanda desde el 31 de agosto al 30 de septiembre
agosto_septiembre = demanda_agosto_2019 + demanda_septiembre_2019
agosto_septiembre = agosto_septiembre[696:]

# Hacemos lo mismo que paar X_train_2, cada dato tiene el dato de la
# demanda 24 y 1 hora antes
X_test_2 = []
for i in range(24, len(agosto_septiembre)):
    list_24_1_before = [agosto_septiembre[i-24], agosto_septiembre[i-1]]
    X_test_2.append(list_24_1_before)
    
# Normalizamos
X_test_2 -= media
X_test_2 /= desviacion
    
# Lista con los precios de septiembre que son los que queremos predecir
precio_septiembre_2019 = precio.loc[lista_septiembre].values
precio_septiembre_2019 = [x[i] for x in precio_septiembre_2019 for i in range(len(x))]

In [12]:
import tensorflow as tf
from tensorflow.keras import layers

# Red neuronal simple, con 4 capas, funcion de activavion ReLu en las
# capas intermedias y linear en la ultima ya que estamos buscando 
# una regresion

model = tf.keras.Sequential([
layers.Dense(256, kernel_initializer='normal', activation='relu', input_dim=2),
layers.Dense(256, kernel_initializer='normal', activation='relu'),
layers.Dense(256, kernel_initializer='normal', activation='relu'),
layers.Dense(1, kernel_initializer='normal', activation='linear')])

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

history = model.fit(X_train_2, y_train_2,
          batch_size=10,
          epochs=6,
          verbose=1)                   )


Train on 23328 samples
Epoch 1/6
23328/23328 [==============================] - 6s 247us/sample - loss: 12.2276 - mean_absolute_error: 12.2276
Epoch 2/6
23328/23328 [==============================] - 6s 247us/sample - loss: 10.1044 - mean_absolute_error: 10.1044
Epoch 3/6
23328/23328 [==============================] - 6s 270us/sample - loss: 10.0993 - mean_absolute_error: 10.0993
Epoch 4/6
23328/23328 [==============================] - 6s 271us/sample - loss: 9.9973 - mean_absolute_error: 9.9973
Epoch 5/6
23328/23328 [==============================] - 6s 273us/sample - loss: 9.9571 - mean_absolute_error: 9.9572
Epoch 6/6
23328/23328 [==============================] - 6s 271us/sample - loss: 9.9674 - mean_absolute_error: 9.9674


In [19]:
# Hacemos las predicciones y vemos que tal ha ido
prediccion = model.predict(X_test_2)
errores = []
error_total = 0
for i in range(len(prediccion)):
    err = precio_septiembre_2019[i]-prediccion[i]
    error_total += err
    errores.append(err)
print("el error cometido es: ", float(error_total/len(errores)))

el error cometido es:  -11.45322036743164


In [14]:
print(prediccion)

[[113.491615]
 [110.29939 ]
 [108.94324 ]
 [107.862465]
 [107.274254]
 [106.80651 ]
 [106.75147 ]
 [107.64476 ]
 [110.705894]
 [115.35325 ]
 [117.778076]
 [119.58816 ]
 [121.41262 ]
 [122.46667 ]
 [121.85691 ]
 [120.285736]
 [119.74579 ]
 [119.08265 ]
 [117.84329 ]
 [116.398254]
 [115.73304 ]
 [116.93301 ]
 [113.92774 ]
 [113.27195 ]
 [112.22364 ]
 [110.83019 ]
 [109.53739 ]
 [108.73058 ]
 [108.914185]
 [109.07374 ]
 [109.4548  ]
 [112.096825]
 [115.92294 ]
 [115.18091 ]
 [116.5799  ]
 [118.06461 ]
 [118.983025]
 [120.7367  ]
 [122.08905 ]
 [122.100914]
 [121.48292 ]
 [121.57799 ]
 [120.869255]
 [119.46691 ]
 [117.50989 ]
 [117.0765  ]
 [119.21964 ]
 [116.165535]
 [114.09858 ]
 [112.77975 ]
 [111.622665]
 [110.90065 ]
 [110.52694 ]
 [109.77437 ]
 [109.43252 ]
 [110.784996]
 [112.796196]
 [114.36204 ]
 [116.63897 ]
 [118.454704]
 [120.78002 ]
 [122.71593 ]
 [123.70545 ]
 [122.71984 ]
 [121.973015]
 [121.58381 ]
 [120.80772 ]
 [119.492065]
 [117.983925]
 [119.36478 ]
 [119.39516 ]
 [116.

In [15]:
print(precio_septiembre_2019)

[108.92, 103.6, 100.91, 102.98, 102.56, 103.14, 99.05, 97.04, 96.52, 97.58, 95.68, 97.01, 97.26, 102.34, 101.66, 97.63, 95.45, 93.24, 92.44, 95.24, 100.27, 108.25, 102.37, 97.17, 92.67, 89.54, 84.43, 82.79, 83.58, 85.04, 94.32, 107.33, 109.67, 107.78, 102.51, 99.79, 96.99, 99.29, 98.27, 98.71, 96.31, 99.17, 104.46, 109.43, 109.83, 106.64, 99.81, 93.88, 92.68, 92.08, 92.41, 91.82, 91.83, 92.5, 100.77, 106.75, 100.93, 104.69, 105.47, 105.54, 107.15, 108.23, 106.11, 105.88, 105.21, 105.97, 106.63, 110.68, 111.26, 109.52, 106.79, 106.1, 106.66, 105.26, 102.09, 104.74, 102.55, 103.48, 101.64, 105.77, 104.69, 105.13, 104.52, 104.51, 104.9, 105.31, 103.87, 101.25, 100.33, 99.1, 96.66, 97.95, 98.98, 100.57, 100.74, 97.8, 98.45, 91.55, 90.46, 90.49, 88.39, 89.06, 93.37, 99.58, 99.33, 98.42, 95.55, 95.01, 96.82, 99.21, 97.03, 97.81, 97.63, 98.54, 96.23, 98.87, 99.91, 102.94, 96.82, 91.99, 91.09, 87.46, 86.75, 86.5, 84.55, 86.2, 89.67, 97.61, 97.04, 96.75, 95.14, 95.43, 98.14, 99.35, 99.53, 99.47